In [ ]:
!pip install soundfile
!pip install h5py pyyaml
import tensorflow as tf
from tensorflow import keras as keras
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import librosa as lr
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
#os.mkdir("Data")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
os.chdir('gdrive')

In [ ]:
os.listdir()

['My Drive', '.Trash']

In [ ]:
#os.chdir('My Drive/Colab_Upload')
os.rmdir('training')

In [ ]:
duration = 10
sr = 16000
total = duration*sr
shape = (313,513)

filenames = os.listdir()
#filenames.pop(filenames.index('.ipynb_checkpoints'))

def get_spectrogram(wav_file):
    spec = np.absolute(lr.stft(wav_file,1024,512))
    spec = np.transpose(spec,(1,0))
    return spec

def load_file(wav_path):
    global sr
    global duration
    global total
    y,_ = lr.load(wav_path,sr=sr,mono=False)
    mixture = lr.to_mono(y)
    source_1 = y[0]
    source_2 = y[1]
    mix_shape = mixture.shape[0]
    if mixture.shape[0] < total:
        mixture,source_1,source_2 = np.pad(mixture,(0,total-mix_shape),'constant'),np.pad(source_1,(0,total-mix_shape),'constant'),np.pad(source_2,(0,total-mix_shape),'constant')
    return mixture[:total],source_1[:total],source_2[:total]

keptout = []
def main():
    train_data = []
    train_label_1 = []
    train_label_2 = []
    for i in filenames:
        k = np.random.binomial(1,0.85)
        if k == 1:
          mixture,source1,source2 = load_file(i)
          train_data.append(get_spectrogram(mixture))
          train_label_1.append(get_spectrogram(source1))
          train_label_2.append(get_spectrogram(source2))
        else:
          keptout.append(i)
    return np.array(train_data),np.array(train_label_1),np.array(train_label_2)

train,label1,label2 = main()

print(train.shape)
print(label1.shape)
print(label2.shape)

(847, 313, 513)
(847, 313, 513)
(847, 313, 513)


In [ ]:
def divide(x):
    return abs(x[0])/abs((x[1]+np.finfo(float).eps))




def construct_custom_model_with_masking(Input_shape):
    Inputs = keras.layers.Input(shape=Input_shape)
    #x = keras.layers.GaussianNoise(1)(Inputs)
    x = keras.layers.CuDNNGRU(1024,return_sequences=True)(Inputs)
    x = keras.layers.CuDNNGRU(1024,return_sequences=True)(x)
    #x = keras.layers.CuDNNGRU(1024,return_sequences=True)(Inputs)
    x = keras.layers.CuDNNGRU(units=1024,return_sequences=True)(x)
    output_1 = keras.layers.Dense(units=Input_shape[1],activation='relu')(x)
    output_2 = keras.layers.Dense(units=Input_shape[1],activation='relu')(x)
    #output_3 = keras.layers.Dense(units=Input_shape[1])(x)
    total_output = keras.layers.Add()([output_1,output_2])
    divided_1 = keras.layers.Lambda(divide)([output_1, total_output])
    divided_2 = keras.layers.Lambda(divide)([output_2, total_output])
#     # divided_3 = keras.layers.Lambda(divide)([output_3, total_output])
    output_1 = keras.layers.Multiply()([Inputs, divided_1])
    output_2 = keras.layers.Multiply()([Inputs, divided_2])
    output_3 = keras.layers.Multiply()([Inputs, divided_1])
    output_4 = keras.layers.Multiply()([Inputs, divided_2])
    # output_3 = keras.layers.Multiply()([Inputs, divided_3])
    custom_model = keras.models.Model([Inputs],[output_1,output_2,output_3,output_4])
    return custom_model

In [ ]:
os.chdir("..")

In [ ]:
os.getcwd()
os.mkdir("training")
os.listdir()

['abjones_5_07.wav',
 'abjones_5_02.wav',
 'abjones_5_03.wav',
 'abjones_5_04.wav',
 'abjones_5_06.wav',
 'abjones_5_05.wav',
 'abjones_5_01.wav',
 'abjones_4_07.wav',
 'abjones_4_08.wav',
 'abjones_4_06.wav',
 'abjones_4_05.wav',
 'abjones_4_01.wav',
 'abjones_4_02.wav',
 'abjones_4_03.wav',
 'abjones_4_04.wav',
 'abjones_3_14.wav',
 'abjones_3_10.wav',
 'abjones_3_11.wav',
 'abjones_3_12.wav',
 'abjones_3_09.wav',
 'abjones_3_06.wav',
 'abjones_3_03.wav',
 'abjones_3_04.wav',
 'abjones_3_02.wav',
 'abjones_2_09.wav',
 'abjones_2_10.wav',
 'abjones_2_11.wav',
 'abjones_2_12.wav',
 'abjones_3_01.wav',
 'abjones_2_08.wav',
 'abjones_2_07.wav',
 'abjones_2_02.wav',
 'abjones_2_03.wav',
 'abjones_2_04.wav',
 'abjones_2_06.wav',
 'abjones_2_05.wav',
 'abjones_2_01.wav',
 'abjones_1_03.wav',
 'abjones_1_04.wav',
 'abjones_1_02.wav',
 'abjones_1_01.wav',
 'amy_10_01.wav',
 'abjones_5_09.wav',
 'abjones_5_08.wav',
 'amy_1_06.wav',
 'amy_1_05.wav',
 'amy_1_03.wav',
 'amy_1_04.wav',
 'amy_1_02.

In [ ]:

checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
custom_model = construct_custom_model_with_masking(train.shape[1:])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
os.chdir('gdrive')

In [ ]:
custom_model.compile(optimizer=tf.train.AdamOptimizer(),loss=[keras.losses.mean_squared_error,keras.losses.mean_squared_error,keras.losses.mean_squared_error,
                                                             keras.losses.mean_squared_error],loss_weights=[1,1,-0.2,-0.2])
summary = custom_model.fit([train],[label1,label2,label2,label1],batch_size=16,epochs=100)
from google.colab import files
os.chdir('/')
custom_model.save_weights("RNN_NN1.h5")
files.download('RNN_NN1.h5')



Instructions for updating:
Use tf.cast instead.
Epoch 1/100
847/847 [==============================] - 19s 23ms/sample - loss: 3.6465 - multiply_loss: 2.6222 - multiply_1_loss: 2.6183 - multiply_2_loss: 4.0956 - multiply_3_loss: 3.8747
Epoch 2/100
847/847 [==============================] - 16s 19ms/sample - loss: 2.5577 - multiply_loss: 2.1966 - multiply_1_loss: 2.1157 - multiply_2_loss: 4.5633 - multiply_3_loss: 4.2099
Epoch 3/100
847/847 [==============================] - 16s 19ms/sample - loss: 2.0405 - multiply_loss: 1.9701 - multiply_1_loss: 1.9158 - multiply_2_loss: 4.7638 - multiply_3_loss: 4.4632
Epoch 4/100
847/847 [==============================] - 17s 20ms/sample - loss: 1.6834 - multiply_loss: 1.8024 - multiply_1_loss: 1.7877 - multiply_2_loss: 4.9144 - multiply_3_loss: 4.6192
Epoch 5/100
847/847 [==============================] - 17s 20ms/sample - loss: 1.5314 - multiply_loss: 1.7690 - multiply_1_loss: 1.6909 - multiply_2_loss: 4.9865 - multiply_3_loss: 4.6558
Epoch 6/100


MessageError: ignored

In [ ]:
custom_model.save_weights("RNN_NN1.h5")

In [ ]:
#total = 16000*10

print(keptout)
os.chdir("/")


new_model = custom_model = construct_custom_model_with_masking((313,513))
new_model.load_weights('RNN_NN1.h5')
#os.chdir('/content')
mixture_mono,sr = lr.load('Test1.wav',sr=16000,mono=True)
mixture_mono = np.pad(mixture_mono,(0,total-mixture_mono.shape[0]),'constant')
mixture_mono = mixture_mono[:total]
mixture_mono_stft = lr.stft(mixture_mono,1024,512)
#mixture_mono_stft= np.transpose(mixture_mono_stft,(1,0))


mixture_mono_stft_mag,mixture_mono_stft_phase = lr.magphase(mixture_mono_stft)
mixture_mono_stft_mag = np.transpose(mixture_mono_stft_mag,(1,0))
train_data = np.array([mixture_mono_stft_mag])

y = new_model.predict(train_data)



i_y_0 = np.transpose(y[0][0],(1,0))*mixture_mono_stft_phase
i_y_1 = np.transpose(y[1][0],(1,0))*mixture_mono_stft_phase



i_y_0 = lr.istft(i_y_0,512)
i_y_1 = lr.istft(i_y_1,512)




sf.write("test12.wav",i_y_0,sr)
sf.write("test2.wav",i_y_1,sr)

['abjones_2_05.wav', 'abjones_2_08.wav', 'abjones_2_09.wav', 'abjones_3_06.wav', 'abjones_3_10.wav', 'abjones_4_01.wav', 'abjones_4_07.wav', 'abjones_5_04.wav', 'amy_10_04.wav', 'amy_11_01.wav', 'amy_11_02.wav', 'amy_13_05.wav', 'amy_13_03.wav', 'amy_16_04.wav', 'amy_16_07.wav', 'amy_16_08.wav', 'amy_2_05.wav', 'amy_3_03.wav', 'amy_3_04.wav', 'amy_4_03.wav', 'amy_4_07.wav', 'amy_4_11.wav', 'amy_5_03.wav', 'amy_5_01.wav', 'amy_5_08.wav', 'amy_6_02.wav', 'amy_6_05.wav', 'amy_8_03.wav', 'Ani_1_01.wav', 'Ani_4_03.wav', 'annar_1_05.wav', 'annar_4_03.wav', 'annar_4_06.wav', 'ariel_1_01.wav', 'ariel_2_04.wav', 'ariel_3_04.wav', 'ariel_4_03.wav', 'ariel_5_01.wav', 'bobon_2_01.wav', 'bobon_3_04.wav', 'bobon_3_06.wav', 'bobon_4_01.wav', 'bobon_4_04.wav', 'bobon_4_03.wav', 'bobon_4_08.wav', 'bobon_5_10.wav', 'bug_1_07.wav', 'bug_2_01.wav', 'bug_3_03.wav', 'bug_3_02.wav', 'bug_4_02.wav', 'bug_5_04.wav', 'bug_5_06.wav', 'davidson_1_04.wav', 'davidson_1_02.wav', 'davidson_2_02.wav', 'davidson_2_05.w